In [1]:
import pandas as pd
import json
import numpy as np
import os
from google.cloud import storage
from io import BytesIO
from pypdf import PdfReader  # You may need to install pypdf if you haven't already
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain
#from langchain.llms import OpenAI
import vertexai
from vertexai.generative_models import GenerativeModel, Part
from langchain_google_vertexai import ChatVertexAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import logging
from dotenv import load_dotenv

In [2]:
load_dotenv()
json_path = os.getenv('json_path') # Variable donde se encuentra la ruta del json con las credenciales de Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = json_path 
logging.basicConfig(filename='error_log.txt', level=logging.ERROR, format='%(asctime)s %(message)s')
bucket_name = 'tfm_javi'

In [41]:
# Función que se encarga de devolver un PDF dado su nombre y el bucket donde está almacenado
def leer_blob_en_memoria(bucket_name, blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    pdf_content = blob.download_as_bytes()
    return pdf_content

# Función que transforma un pdf en una cadena de texto que puede ser procesada
def extraer_texto_de_pdf_bytes(pdf_bytes):
    pdf_file = BytesIO(pdf_bytes)
    reader = PdfReader(pdf_file)
    text = ''
    for page in reader.pages:
        text += page.extract_text() + '\n'
    return text

# Función que combina las dos funciones creadas anteriormente en una sola
def procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name):
    pdf_bytes = leer_blob_en_memoria(bucket_name, blob_name)
    content = extraer_texto_de_pdf_bytes(pdf_bytes)
    return content

# Función que se encarga de devolver una lista con todos los nombres de los archivos en un bucket
def listar_pdfs(bucket_name):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blobs = bucket.list_blobs()

    # Filtra los archivos PDF y guarda sus URLs completas en la lista
    listado_pdfs = [f"gs://{bucket_name}/{blob.name}" for blob in blobs if blob.name.endswith('.pdf') and not blob.name.startswith('resoluciones/')]
    return listado_pdfs

# Función que se encarga de dividir la lista de todos los PDFs en 5 partes para facilitar su procesado en tandas
def dividir_lista_pdfs(listado_pdfs):
    total = len(listado_pdfs)
    tamaño_parte = total // 5
    
    parte1 = listado_pdfs[:tamaño_parte]
    parte2 = listado_pdfs[tamaño_parte:2*tamaño_parte]
    parte3 = listado_pdfs[2*tamaño_parte:3*tamaño_parte]
    parte4 = listado_pdfs[3*tamaño_parte:4*tamaño_parte]
    parte5 = listado_pdfs[4*tamaño_parte:]
    
    return parte1, parte2, parte3, parte4, parte5

In [7]:
# Definición los esquemas de respuesta
response_schemas = [
    ResponseSchema(name="numero_expediente", description="Número de Expediente, ejemplo SD2023/0000046"),
    ResponseSchema(name="resolucion", description="Una de las siguientes opciones: negada_con_oposicion, negada_sin_oposicion, aprobada_sin_oposicion, aprobada_con_oposición"),
    ResponseSchema(name="numero_de_resolución", description="Número entero de la resolución, ejemplo 2195"),
    ResponseSchema(name="denominacion", description="Nombre de la empresa que solicita el registro de la marca"),
    ResponseSchema(name="vigencia", description="Fecha en que expira la vigencia del registro, o texto si es negada o vencida"),
    ResponseSchema(name="titular", description="Titular de la marca que intenta registrar"),
    ResponseSchema(name="clase", description="Número o lista de números de la Clasificación Internacional de Niza, por ejemplo [42, 35, 27]"),
    ResponseSchema(name="gaceta", description="Número de la gaceta de Propiedad Industrial donde se publica"),
    ResponseSchema(name="tipo", description="Tipo de registro que se intenta realizar, por ejemplo Mixta, Nominativa, Figurativa"),
    ResponseSchema(name="fecha_solicitud", description="Fecha de presentación de la solicitud"),
    ResponseSchema(name="fecha_resolucion", description="Fecha de resolución"),
    ResponseSchema(name="nombre_opositor", description="Nombre de la empresa que se opone a la publicación"),
    ResponseSchema(name="signo_opositor_opositores", description="Signo o signos de los opositores en conflicto"),
    ResponseSchema(name="argumento_oposición", description="Argumentos en los que se basa para oponerse al registro y artículos en los que se apoya"),
    ResponseSchema(name="explicacion_argumentos_oposicion", description="Breve resumen y explicación de los argumentos de la oposición"),
    ResponseSchema(name="resolucion_organismo", description="Resolución del organismo competente, por ejemplo: 'DENIEGA el registro de la marca PAPELES LA FAVORITA (Mixta)'")
]


In [8]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
# Escapar las llaves para evitar la interpretación de variables
format_instructions = format_instructions.replace("{", "{{").replace("}", "}}")

In [9]:
# Definición de los prompts que se le van a pasar a VertexAI para darle instrucciones de qué hacer
system_prompt = """
Eres un Experto abogado Colombiano en analizar resoluciones del SIC (Superintendencia de Industria y Comercio de Colombia),en el ámbito de registro de marcas y lemas.
quiero que extraigas el numero de Expediente, la resolución del conflicto, el numero de la resolución, el nombre de la marca que intenta registrarse, el titular que intenta registrar la marca, el numero de clase que intenta registrar, 
el numero de la gaceta en que ha sido publicada, la fecha de solicitud de registro, nombre de la empresa opositora, el titular de la empresa que se opone si aparece, y los argumentos de derecho en los que se apoya el opositor.
"""
human_prompt = f"""Extrae la información indicada en DATOS a partir del TEXTO de la resolución

TEXT
---
\n\n{{contenido_pdf}}
---

DATOS
{format_instructions}
"""

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template(human_prompt),
])

In [11]:
# Define tu LLM 
llm = ChatVertexAI(
    model="gemini-1.5-pro-001"
      #verificar la relacion megas tokens  
)

chain = LLMChain(llm=llm, prompt=prompt, output_parser=output_parser)

In [38]:
# Obtener la lista de PDFs
listado_pdfs = listar_pdfs(bucket_name)

# Comprobamos la longitud total de los PDFs en el bucket
len(listado_pdfs)

In [43]:
# Dividimos la lista de PDFs en 5 partes para facilitar su procesado (procesar tantos PDFs lleva tiempo y procesar 12000-13000 PDFs implicaria demasiado tiempo)
parte1, parte2, parte3, parte4, parte5 = dividir_lista_pdfs(listado_pdfs)

In [51]:
# Inicializar una lista para almacenar los resultados y los errores
resultados = []
listado_pdfs_error = []

## Parte 1

A continuación se va a proceder a transformar todos los PDFs de la parte 1 en datos que puedan ser explotados mas adelante. Durante el procesado de estos PDFs a veces se pueden dar errores, por lo que guardamos los nombres de los archivos que han dado error para poder procesarlos de nuevo mas adelante. Una vez se haya concluido de procesar la parte 1, se hará lo mismo con el resto de las partes (2, 3, 4, y 5)

In [59]:
# Procesamos todos los PDFs de la primera parte
for pdf_uri in parte1:
    blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
    contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
    # Verificar si el contenido del PDF no está vacío
    if contenido_pdf.strip():
        try:
            # Invocar la cadena con el contenido del PDF
            res = chain.invoke({"contenido_pdf": contenido_pdf})
            #Agregar el resultado a la lista de resultados
            resultados.append(res)
            #Imprimir el resultado
            print(f"Resultado para {blob_name}:\n{res}\n")
        except Exception as e:
            # Si hay algún error al procesar el archivo (a veces pasa), guardamos su nombre para intentarlo otra vez mas adelante
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            listado_pdfs_error.append(blob_name)
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
    else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

NotFound: 404 GET https://storage.googleapis.com/download/storage/v1/b/tfm_javi/o/g?alt=media: No such object: tfm_javi/g: ('Request failed with status code', 404, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PARTIAL_CONTENT: 206>)

## Errores de analisis de la parte 1

Todos los PDFs que dieron error al ser procesados en la sección anterior se han fuardado en una variable llamada listado_pdfs_error. Para procesar esta parte procedemos a llamar a la misma función que en el apartado anterior

In [57]:
# Comprobamos la longitud de la lista con los PDFs que han dado error al ser procesados
print(listado_pdfs_error)    

In [97]:
# Recorremos la lista de nuevo procesando los PDFs una segunda vez
for pdf_uri in listado_pdfs_error:
     blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
     contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
      # Verificar si el contenido del PDF no está vacío
     if contenido_pdf.strip():
         try:
              # Invocar la cadena con el contenido del PDF
             res = chain.invoke({"contenido_pdf": contenido_pdf})
              #Agregar el resultado a la lista de resultados
             resultados.append(res)
              #Imprimir el resultado
             print(f"Resultado para {blob_name}:\n{res}\n")
         except Exception as e:
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
     else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

Resultado para resoluciones/NCO-SD2022-0073142.pdf:
{'contenido_pdf': 'REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE INDUSTRIA Y COMERCIO \nResolución N° 16207\nRef. Expediente N° SD2022/0073142\nRef. Reg. Internacional N° 1673357\nPágina 1 de 48Por la cual se decide una solicitud de registro \nLA DIRECTORA DE SIGNOS DISTINTIVOS (E)\nen ejercicio de sus facultades legales y,\nCONSIDERANDO\nQue por escrito presentado ante la Oficina Internacional el día 30 de marzo de 2022 y \nradicada ante la Superintendencia de Industria y Comercio el día 21 de julio de 2022, YS \nFAMILY MAYORISTA 2021 SLU, solicitó el registro de la marca FAMILY (Mixta) para \ndistinguir productos comprendidos en las clases 8, 20, 21 y 24 de la Clasificación \nInternacional de Niza1. \nQue publicado en la Gaceta de Propiedad Industrial No. 974, PRODUCTOS FAMILIA S.A., \npresentó oposición en contra de las clases 20, 21 y 24 con fundamento en las causales \nde irregistrabilidad establecidas en los literales a) y h) del a

Parece que esta vez si que ha procesado bien los tres archivos que diéron error, vamos a construir un df con los resultados

In [60]:
df_resultados = pd.DataFrame(resultados)

In [61]:
df_resultados

,contenido_pdf,text
0,REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE IN...,"{'numero_expediente': '14258446', 'resolucion'..."
1,REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE IND...,"{'numero_expediente': '15006549', 'resolucion'..."
2,REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE IND...,"{'numero_expediente': '15030961', 'resolucion'..."
3,REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE IND...,"{'numero_expediente': '15036335', 'resolucion'..."
4,REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE IND...,"{'numero_expediente': '15036379', 'resolucion'..."
...,...,...
2551,REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE IND...,"{'numero_expediente': 'SD2019/0040431', 'resol..."
2552,REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE IND...,"{'numero_expediente': 'SD2021/0031995', 'resol..."
2553,REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE IND...,"{'numero_expediente': 'SD2021/0085336', 'resol..."
2554,REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE IND...,"{'numero_expediente': 'SD2017/0025025', 'resol..."


In [62]:
pd.set_option('display.max_columns', None)

In [63]:
df_ai_annotations_full_1 = pd.DataFrame.from_records(df_resultados['text'])

In [64]:
df_ai_annotations_full_1

,numero_expediente,resolucion,numero_de_resolución,denominacion,vigencia,titular,clase,gaceta,tipo,fecha_solicitud,fecha_resolucion,nombre_opositor,signo_opositor_opositores,argumento_oposición,explicacion_argumentos_oposicion,resolucion_organismo
0,14258446,aprobada_con_oposición,82711,ChaCheer,20 de marzo de 2023.,"QIAQIA FOOD CO., LTD.","[29, 30]",719,Mixta,24 de noviembre de 2014,29 de noviembre de 2016,INVERSIONES CHEERS DEL CARIBE S.A.,CHEERS SINCE 2009,Basada en el literal a) del artículo 136 de la...,La oposición argumenta que existe un alto grad...,Declara infundada la oposición interpuesta por...
1,15006549,aprobada_con_oposición,50676,Granini,26 de agosto de 2030,ECKES-GRANINI GROUP GMBH,32,726,Mixta,14 de enero de 2015,26 de agosto de 2020,TORRECAFE AGUILA ROJA [&] CIA S.A.,GRANINO\n(Nominativa),La marca solicitada GRANINI es confundible con...,El opositor argumenta que existe riesgo de con...,Se declara infundada la oposición interpuesta ...
2,15030961,aprobada_con_oposición,3429,SECTRA,16 de mayo de 2024,SECTRA AB,"[9, 10, 16, 35, 38, 41, 42, 44]",755,Nominativa,12 de febrero de 2015,3 de febrero de 2017,"ESPECIALISTAS EN RADIOLOGIA S.A.S, SIXTRA CHIL...","ESPECTRA, SIXTRA",Los opositores argumentan que la marca solicit...,Los opositores consideran que existe un alto r...,La Superintendencia de Industria y Comercio de...
3,15036335,aprobada_con_oposición,19923,BODEGAS RODA SELA,26 de junio de 2022,"BODEGAS RODA, S.A.",33,767,Mixta,5 de diciembre de 2014,24 de abril de 2017,BODEGAS DE MOSELA LTDA.,BODEGAS DE MOSELA (Nominativa),El opositor argumenta que el signo solicitado ...,La oposición se fundamenta en la similitud ent...,La Superintendencia de Industria y Comercio re...
4,15036379,aprobada_con_oposición,10639,bag,29 de octubre de 2024,J. Luscombe Associates Limited,"[9, 14, 18, 25, 35]",736,Mixta,19 de febrero de 2015,8 de marzo de 2017,OUR BAG S.A.S.,OUR BAG y b. bag,Artículo 136 literal a) de la Decisión 486 de ...,El opositor argumenta que la marca solicitada ...,Se declara infundada la oposición y se concede...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2551,SD2019/0040431,aprobada_con_oposición,220779,MI WALLET,20 de noviembre de 2028,XIAOMI INC.,"[9, 36]",863,Mixta,20 de noviembre de 2018,17 de octubre de 2019,UALET S.A. COMISIONISTA DE BOLSA,UALET,La oposición se basa en las causales de irregi...,El opositor argumenta que MI WALLET es similar...,Se declara INFUNDADA la oposición interpuesta ...
2552,SD2021/0031995,aprobada_con_oposición,2853,ALPECIN HYBRID,15 de mayo de 2028,DR. KURT WOLFF GMBH & CO. KG,"[3, 5]",929,Mixta,17 de marzo de 2021,31 de enero de 2022,GLAUKOS CORPORATION,IBRID (Nominativo),La marca solicitada ALPECIN HYBRID es similar ...,El opositor argumenta que la similitud fonétic...,Se declara infundada la oposición interpuesta ...
2553,SD2021/0085336,aprobada_con_oposición,82395,Fini,04/03/2031,"Sánchez Cano, S.A.","[5, 29, 30, 35, 36, 41]",963,Mixta,04/03/2021,23/11/2022,Finppi Colombia SA.S.\nAlimentos Finca S.A.S.,"Finppi, FINCA MININO, FINCA MIRRINGO, F FINCA,...",Los opositores argumentan que la marca solicit...,Los argumentos de la oposición se basan en la ...,Se deniega la extensión de la notoriedad de la...
2554,SD2017/0025025,aprobada_sin_oposicion,64235,MR-174 Watching the World \n. Designed by Mits...,12 de agosto de 2019,"MITSUI CHEMICALS, INC.",9,None,Mixta,None,9 de octubre de 2017,None,None,None,None,Conceder el registro de la Marca MR-174 Watchi...


In [65]:
# Comprobamos cuantas files tienen entradas en blanco y el tipo de variable de cada columna
df_ai_annotations_full_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556 entries, 0 to 2555
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   numero_expediente                 2556 non-null   object
 1   resolucion                        2556 non-null   object
 2   numero_de_resolución              2556 non-null   object
 3   denominacion                      2404 non-null   object
 4   vigencia                          2535 non-null   object
 5   titular                           2556 non-null   object
 6   clase                             2555 non-null   object
 7   gaceta                            588 non-null    object
 8   tipo                              2556 non-null   object
 9   fecha_solicitud                   1399 non-null   object
 10  fecha_resolucion                  2550 non-null   object
 11  nombre_opositor                   584 non-null    object
 12  signo_opositor_oposi

In [67]:
# Guardamos los resultados en un archivo csv
df_ai_annotations_full_1.to_csv("datasetia_full_1.csv")

A continuación vamos a realizar los mismo pasos para las partes 2, 3, 4, y 5

## Parte 2

In [ ]:
# Reiniciamos las listas
resultados = []
listado_pdfs_error = []

In [ ]:
# Procesamos todos los PDFs de la segunda parte
for pdf_uri in parte2:
    blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
    contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
    # Verificar si el contenido del PDF no está vacío
    if contenido_pdf.strip():
        try:
            # Invocar la cadena con el contenido del PDF
            res = chain.invoke({"contenido_pdf": contenido_pdf})
            #Agregar el resultado a la lista de resultados
            resultados.append(res)
            #Imprimir el resultado
            print(f"Resultado para {blob_name}:\n{res}\n")
        except Exception as e:
            # Si hay algún error al procesar el archivo (a veces pasa), guardamos su nombre para intentarlo otra vez mas adelante
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            listado_pdfs_error.append(blob_name)
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
    else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

## Errores de analisis de la parte 2

In [ ]:
# Comprobamos la longitud de la lista con los PDFs que han dado error al ser procesados
print(listado_pdfs_error)    

In [ ]:
# Recorremos la lista de nuevo procesando los PDFs una segunda vez
for pdf_uri in listado_pdfs_error:
     blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
     contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
      # Verificar si el contenido del PDF no está vacío
     if contenido_pdf.strip():
         try:
              # Invocar la cadena con el contenido del PDF
             res = chain.invoke({"contenido_pdf": contenido_pdf})
              #Agregar el resultado a la lista de resultados
             resultados.append(res)
              #Imprimir el resultado
             print(f"Resultado para {blob_name}:\n{res}\n")
         except Exception as e:
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
     else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

In [ ]:
df_resultados = pd.DataFrame(resultados)
pd.set_option('display.max_columns', None)
df_ai_annotations_full_2 = pd.DataFrame.from_records(df_resultados['text'])

In [ ]:
# Comprobamos cuantas files tienen entradas en blanco y el tipo de variable de cada columna
df_ai_annotations_full_2.info()

In [ ]:
# Guardamos los resultados en un archivo csv
df_ai_annotations_full_2.to_csv("datasetia_full_2.csv")

## Parte 3


In [ ]:
# Reiniciamos las listas
resultados = []
listado_pdfs_error = []

In [ ]:
# Procesamos todos los PDFs de la segunda parte
for pdf_uri in parte3:
    blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
    contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
    # Verificar si el contenido del PDF no está vacío
    if contenido_pdf.strip():
        try:
            # Invocar la cadena con el contenido del PDF
            res = chain.invoke({"contenido_pdf": contenido_pdf})
            #Agregar el resultado a la lista de resultados
            resultados.append(res)
            #Imprimir el resultado
            print(f"Resultado para {blob_name}:\n{res}\n")
        except Exception as e:
            # Si hay algún error al procesar el archivo (a veces pasa), guardamos su nombre para intentarlo otra vez mas adelante
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            listado_pdfs_error.append(blob_name)
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
    else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

## Errores de analisis de la parte 3

In [ ]:
# Comprobamos la longitud de la lista con los PDFs que han dado error al ser procesados
print(listado_pdfs_error)    

In [ ]:
# Recorremos la lista de nuevo procesando los PDFs una segunda vez
for pdf_uri in listado_pdfs_error:
     blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
     contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
      # Verificar si el contenido del PDF no está vacío
     if contenido_pdf.strip():
         try:
              # Invocar la cadena con el contenido del PDF
             res = chain.invoke({"contenido_pdf": contenido_pdf})
              #Agregar el resultado a la lista de resultados
             resultados.append(res)
              #Imprimir el resultado
             print(f"Resultado para {blob_name}:\n{res}\n")
         except Exception as e:
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
     else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

In [ ]:
df_resultados = pd.DataFrame(resultados)
pd.set_option('display.max_columns', None)
df_ai_annotations_full_3 = pd.DataFrame.from_records(df_resultados['text'])

In [ ]:
# Comprobamos cuantas files tienen entradas en blanco y el tipo de variable de cada columna
df_ai_annotations_full_3.info()

In [ ]:
# Guardamos los resultados en un archivo csv
df_ai_annotations_full_3.to_csv("datasetia_full_3.csv")

## Parte 4

In [ ]:
# Reiniciamos las listas
resultados = []
listado_pdfs_error = []

In [ ]:
# Procesamos todos los PDFs de la segunda parte
for pdf_uri in parte4:
    blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
    contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
    # Verificar si el contenido del PDF no está vacío
    if contenido_pdf.strip():
        try:
            # Invocar la cadena con el contenido del PDF
            res = chain.invoke({"contenido_pdf": contenido_pdf})
            #Agregar el resultado a la lista de resultados
            resultados.append(res)
            #Imprimir el resultado
            print(f"Resultado para {blob_name}:\n{res}\n")
        except Exception as e:
            # Si hay algún error al procesar el archivo (a veces pasa), guardamos su nombre para intentarlo otra vez mas adelante
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            listado_pdfs_error.append(blob_name)
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
    else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

## Errores de analisis de la parte 4

In [ ]:
# Comprobamos la longitud de la lista con los PDFs que han dado error al ser procesados
print(listado_pdfs_error)   

In [ ]:
# Recorremos la lista de nuevo procesando los PDFs una segunda vez
for pdf_uri in listado_pdfs_error:
     blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
     contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
      # Verificar si el contenido del PDF no está vacío
     if contenido_pdf.strip():
         try:
              # Invocar la cadena con el contenido del PDF
             res = chain.invoke({"contenido_pdf": contenido_pdf})
              #Agregar el resultado a la lista de resultados
             resultados.append(res)
              #Imprimir el resultado
             print(f"Resultado para {blob_name}:\n{res}\n")
         except Exception as e:
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
     else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

In [ ]:
df_resultados = pd.DataFrame(resultados)
pd.set_option('display.max_columns', None)
df_ai_annotations_full_4 = pd.DataFrame.from_records(df_resultados['text'])

In [ ]:
# Comprobamos cuantas files tienen entradas en blanco y el tipo de variable de cada columna
df_ai_annotations_full_4.info()

In [ ]:
# Guardamos los resultados en un archivo csv
df_ai_annotations_full_4.to_csv("datasetia_full_4.csv")

## Parte 5

In [ ]:
# Reiniciamos las listas
resultados = []
listado_pdfs_error = []

In [ ]:
# Procesamos todos los PDFs de la segunda parte
for pdf_uri in parte5:
    blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
    contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
    # Verificar si el contenido del PDF no está vacío
    if contenido_pdf.strip():
        try:
            # Invocar la cadena con el contenido del PDF
            res = chain.invoke({"contenido_pdf": contenido_pdf})
            #Agregar el resultado a la lista de resultados
            resultados.append(res)
            #Imprimir el resultado
            print(f"Resultado para {blob_name}:\n{res}\n")
        except Exception as e:
            # Si hay algún error al procesar el archivo (a veces pasa), guardamos su nombre para intentarlo otra vez mas adelante
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            listado_pdfs_error.append(blob_name)
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
    else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

## Errores de analisis de la parte 5

In [ ]:
# Comprobamos la longitud de la lista con los PDFs que han dado error al ser procesados
print(listado_pdfs_error)   

In [ ]:
# Recorremos la lista de nuevo procesando los PDFs una segunda vez
for pdf_uri in listado_pdfs_error:
     blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
     contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
      # Verificar si el contenido del PDF no está vacío
     if contenido_pdf.strip():
         try:
              # Invocar la cadena con el contenido del PDF
             res = chain.invoke({"contenido_pdf": contenido_pdf})
              #Agregar el resultado a la lista de resultados
             resultados.append(res)
              #Imprimir el resultado
             print(f"Resultado para {blob_name}:\n{res}\n")
         except Exception as e:
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
     else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

In [ ]:
df_resultados = pd.DataFrame(resultados)
pd.set_option('display.max_columns', None)
df_ai_annotations_full_5 = pd.DataFrame.from_records(df_resultados['text'])

In [ ]:
# Comprobamos cuantas files tienen entradas en blanco y el tipo de variable de cada columna
df_ai_annotations_full_5.info()

In [ ]:
# Guardamos los resultados en un archivo csv
df_ai_annotations_full_5.to_csv("datasetia_full_5.csv")